In [1]:
%load_ext autoreload
%autoreload 2

In [54]:
# Standard library imports.
import datetime
import itertools
import json

# Related third party imports.
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import tqdm

# Local application/library specific imports.
from src.models.network_simulation import NetworkSimulation
from src.data.json_encoder import NumpyEncoder

def add_cumulative(df):
    for sim_id in df['simulation_id'].unique():
        mask = df['simulation_id'] == sim_id
        df.loc[mask,'acceptable_count_cum'] = df.loc[mask,'acceptable_count'].cumsum()
        df.loc[mask,'vSDN_count_cum'] = df.loc[mask,'vSDN_count'].cumsum()
        df.loc[mask,'acceptance_ratio_cum'] = df.loc[mask,'acceptable_count_cum'] / df.loc[mask,'vSDN_count_cum']

In [64]:
network_name = '25_italy'
max_vSDN_size = 10
hp_type = 'ilp'
hp_objective = 'acceptance ratio'

possible_settings = {
    'network_name': [network_name],
    'latency_factor': [0.5], # np.arange(0.4, 0.7, 0.1),
    'shortest_k': [16],
    'hp_type': [hp_type],
    'hp_objective': [hp_objective],
    'sim_repeat': [3],
    'timesteps': [10],
    'max_request_size': [max_vSDN_size],
    'request_per_timestep': [5],
    'TTL_range': [5],
    'vSDN_count_ilp':[100],
    'vSDN_size_ilp':[max(2, int(max_vSDN_size / 2))]
}
param_names_1 = list(possible_settings.keys())
setting_list = [
    dict(zip(param_names_1, x))
    for x in itertools.product(*possible_settings.values())
]

In [ ]:
# Normal Simulation
setting = setting_list[0]

ns = NetworkSimulation(**setting)
ns.init_simulation(**setting)
ns.run_multiple_dynamic_simulations(**setting)
run_log = ns.get_logs()
df = pd.DataFrame(run_log)
add_cumulative(df)

In [66]:
df[['hp_runtime', 'acceptance_ratio', 'acceptance_ratio_cum', 'active_vSDN_count', 'mean_switch_hpair_count', 'simulation_id', 'timestep']]

,hp_runtime,acceptance_ratio,acceptance_ratio_cum,active_vSDN_count,mean_switch_hpair_count,simulation_id,timestep
0,82.32,0.50,0.50,2,57.20,0,1
1,82.32,0.25,0.38,2,43.32,0,2
2,82.32,0.50,0.42,4,30.20,0,3
3,82.32,0.25,0.38,5,22.68,0,4
4,82.32,0.50,0.40,5,25.08,0,5
5,82.32,0.25,0.38,3,27.20,0,6
6,82.32,0.25,0.36,4,25.56,0,7
7,82.32,0.75,0.41,4,42.32,0,8
8,82.32,0.75,0.44,4,38.88,0,9
9,82.32,0.25,0.42,2,39.68,0,10


In [69]:
df.groupby(by='timestep').mean()[['acceptance_ratio_cum', 'active_vSDN_count']]

,acceptance_ratio_cum,active_vSDN_count
timestep,,
1,0.50,2.00
2,0.50,3.33
3,0.50,5.33
4,0.50,5.00
5,0.47,3.67
6,0.47,4.33
7,0.45,4.33
8,0.46,3.67
9,0.47,4.00


In [ ]:
# Simulation
simulation_logs = []
for setting in tqdm.tqdm(setting_list, total=len(setting_list)):
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.run_dynamic_simulation(**setting)
    simulation_logs.extend(ns.get_logs())

In [ ]:
# Optimal Run
setting = setting_list[0]

ns = NetworkSimulation(**setting)
ns.init_simulation(**setting)
ns.run_optimal_dynamic_simulation(**setting)
run_log_opt = ns.get_logs()
df_opt = pd.DataFrame(run_log_opt)
add_cumulative(df_opt)

In [32]:
df_opt[['hp_runtime', 'acceptance_ratio', 'acceptance_ratio_cum', 'active_vSDN_count', 'mean_switch_hpair_count', 'hp_changed']]

,hp_runtime,acceptance_ratio,acceptance_ratio_cum,active_vSDN_count,mean_switch_hpair_count,hp_changed
0,1.92,1.00,1.00,4,95.96,0
1,1.92,1.00,1.00,5,78.48,0
2,1.92,1.00,1.00,8,52.24,0
3,1.92,1.00,1.00,11,34.36,0
4,1.92,1.00,1.00,11,35.44,0
...,...,...,...,...,...,...
95,1.92,1.00,1.00,11,36.08,0
96,1.92,1.00,1.00,13,35.08,0
97,1.92,1.00,1.00,12,34.44,0
98,1.92,1.00,1.00,10,35.28,0


In [33]:
pd.concat([df[['hp_runtime', 'acceptance_ratio_cum', 'active_vSDN_count', 'mean_switch_hpair_count', 'h_list']],df_opt[['hp_runtime', 'acceptance_ratio_cum', 'active_vSDN_count', 'mean_switch_hpair_count', 'h_list', 'hp_changed']]], axis=1).sort_index(axis=1)

,acceptance_ratio_cum,acceptance_ratio_cum,active_vSDN_count,active_vSDN_count,h_list,h_list,hp_changed,hp_runtime,hp_runtime,mean_switch_hpair_count,mean_switch_hpair_count
0,1.00,1.00,4,4,"[0, 20, 23, 24]","[0, 20, 21, 23]",0,12.70,1.92,95.96,95.96
1,1.00,1.00,5,5,"[0, 20, 23, 24]","[0, 20, 21, 23]",0,12.70,1.92,78.48,78.48
2,1.00,1.00,8,8,"[0, 20, 23, 24]","[0, 20, 21, 23]",0,12.70,1.92,52.24,52.24
3,1.00,1.00,11,11,"[0, 20, 23, 24]","[0, 20, 21, 23]",0,12.70,1.92,34.36,34.36
4,1.00,1.00,11,11,"[0, 20, 23, 24]","[0, 20, 21, 23]",0,12.70,1.92,34.28,35.44
...,...,...,...,...,...,...,...,...,...,...,...
95,1.00,1.00,11,11,"[0, 20, 23, 24]","[0, 20, 21, 23]",0,12.70,1.92,36.12,36.08
96,1.00,1.00,13,13,"[0, 20, 23, 24]","[0, 20, 21, 23]",0,12.70,1.92,36.12,35.08
97,1.00,1.00,12,12,"[0, 20, 23, 24]","[0, 20, 21, 23]",0,12.70,1.92,38.36,34.44
98,1.00,1.00,10,10,"[0, 20, 23, 24]","[0, 20, 21, 23]",0,12.70,1.92,38.80,35.28


In [ ]:
# Baseline Simulation
simulation_logs = []
for setting in tqdm.tqdm(setting_generator, total=len(setting_generator)):
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.baseline_simulation(possible_request_settings=possible_request_settings, **setting)
    simulation_logs.extend(ns.get_logs())

In [ ]:
with open(f"../results/{network_name}/{datetime.date.today()}-{network_name}-test.json", 'w') as file:
    json.dump(simulation_logs, file, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)

# Analyse simulation logs

In [ ]:
df_heu = pd.DataFrame(simulation_logs)

In [ ]:
df_heu.query("latency_factor == 0.5 & vSDN_size == 2")[['hp_runtime', 'acceptance_ratio']]#.mean()

In [5]:
df_ilp = pd.DataFrame(simulation_logs)

In [6]:
df_ilp.query("latency_factor == 0.4 & vSDN_size == 3")[['hp_runtime', 'acceptable_count','acceptance_ratio', 'acceptance_ratio_ilp', 'h_list']]#.mean()

,hp_runtime,acceptable_count,acceptance_ratio,acceptance_ratio_ilp,h_list
1,0.73,36,0.580645,None,"[3, 6, 12, 15, 18, 19, 20, 23]"
9,0.61,33,0.532258,None,"[3, 6, 12, 15, 18, 19, 20, 23]"
17,0.64,32,0.516129,None,"[3, 6, 12, 15, 18, 19, 20, 23]"
25,0.69,31,0.500000,None,"[3, 6, 12, 15, 18, 19, 20, 23]"


In [ ]:
df_opt = pd.DataFrame(simulation_logs)

In [ ]:
df_opt.query("latency_factor == 0.4 & vSDN_size == 2")[['hp_runtime', 'acceptance_ratio', 'acceptance_ratio_ilp']]#.mean()